In [ ]:
import os
import numpy as np
from shapely.geometry import box
from pyproj import Proj, transform
import requests
from io import BytesIO
from PIL import Image

# Define your AOI (Area of Interest)
aoi = box(10.0, 20.0, 30.0, 40.0)  # Example bounding box

# Convert AOI to geographic coordinates
in_proj = Proj(init='epsg:4326')
out_proj = Proj(init='epsg:3411')  # NSIDC projection
min_lon, min_lat, max_lon, max_lat = aoi.bounds
min_x, min_y = transform(in_proj, out_proj, min_lon, min_lat)
max_x, max_y = transform(in_proj, out_proj, max_lon, max_lat)
bounding_box = f'{min_x},{min_y},{max_x},{max_y}'

# NSIDC API endpoint and parameters
api_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'
short_name = 'NSIDC-0729_2'
version = '002'
time_start = '2024-01-01T00:00:00Z'
time_end = '2024-01-31T23:59:59Z'
bounding_box = bounding_box
format_ = 'GeoTIFF'
projection = 'GEOGRAPHIC'
size = '1000,1000'
email = '@example.com'
agent = 'Python'

# Construct NSIDC API request
request_params = {
    'short_name': short_name,
    'version': version,
    'time': f'{time_start}/{time_end}',
    'bounding_box': bounding_box,
    'format': format_,
    'projection': projection,
    'size': size,
    'email': email,
    'agent': agent
}

# Send request to NSIDC API
response = requests.get(api_url, params=request_params)

# Check if request is successful
if response.status_code == 200:
    # Save retrieved data
    with open('snow_depth_data.tif', 'wb') as f:
        f.write(response.content)
    print('Snow depth data downloaded successfully.')

    # Example: Convert GeoTIFF to numpy array
    snow_depth_image = Image.open(BytesIO(response.content))
    snow_depth_array = np.array(snow_depth_image)

    # Example: Thresholding to create ground truth data
    snow_threshold = 100  # Adjust threshold as needed
    ground_truth = np.where(snow_depth_array > snow_threshold, 1, 0)
    np.save('ground_truth.npy', ground_truth)

    print('Ground truth data created successfully.')

else:
    print('Failed to retrieve snow depth data.')
